In [1]:
import pandas as pd
from sklearn import *


In [2]:
path = '/Users/rosreen/Desktop/oracle-dbt/seeds/'
file = path + 'invoices.csv'

data = pd.read_csv(file)
data.head()

/var/folders/8v/s5jwzy2568j12y51zbwc4b8w0000gq/T/ipykernel_64514/1117076995.py:4: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file)


,Unnamed: 0,SOURCE_RECORD_ID,INVOICE_ROUTING_ATTR_3,PAYMENT_REASON_COMMENTS,INVOICE_CANCELLED_FLAG,SECOND_DISCOUNT_LEDGER_AMOUNT,INVOICE_VALIDATION_STATUS_CODE,REMAINING_GLOBAL_AMOUNT,PRE_POST_SLA_FLAG,HEADER_UPDATED_BY_USER_ID,...,INVOICED_ON_DATE,INVOICE_RECEIVED_DATE,INVOICE_SCHEDULE_DUE_DATE,FIRST_DISCOUNT_DATE,SECOND_DISCOUNT_DATE,THIRD_DISCOUNT_DATE,INVOICE_CREATION_DATE_1,INVOICE_ACCOUNTING_DATE,INVC_SCHEDULE_CLEARED_DATE,INVOICE_CREATION_DATE
0,0,366287~1,NaN,NaN,NO,0.0,APPROVED,636.493600,PRE,AP_SUPV_PD,...,11/16/18,1/1/01,11/16/18,1/1/01,1/1/01,1/1/01,11/16/18,11/16/18,53:00.0,18:48.0
1,1,374072~1,NaN,NaN,NO,0.0,APPROVED,0.000000,PRE,AP_SUPV_PD,...,11/19/18,1/1/01,11/19/18,1/1/01,1/1/01,1/1/01,11/19/18,11/19/18,59:55.0,50:46.0
2,2,368123~1,NaN,NaN,NO,0.0,APPROVED,3279.814080,PRE,AP_SUPV_PD,...,11/19/18,1/1/01,11/19/18,1/1/01,1/1/01,1/1/01,11/19/18,11/19/18,53:00.0,21:08.0
3,3,248831~1,NaN,NaN,NO,0.0,APPROVED,3494.252874,POST,AP_SUPV_ITALY,...,11/20/06,1/1/01,11/20/06,1/1/01,1/1/01,1/1/01,12/8/15,11/20/06,53:00.0,07:33.0
4,4,366184~1,NaN,NaN,NO,0.0,APPROVED,3624.269440,PRE,AP_SUPV_PD,...,11/16/18,1/1/01,11/16/18,1/1/01,1/1/01,1/1/01,11/16/18,11/16/18,53:00.0,36:57.0
